# Evaluation of the Model

## Prerequisite Installations

In [1]:
!pip install embedding_as_service

     |████████████████████████████████| 140 kB 5.3 MB/s 
     |████████████████████████████████| 110.5 MB 39 kB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 312 kB 52.2 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 17.3 MB 128 kB/s 
     |████████████████████████████████| 57 kB 5.5 MB/s 
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 1.0 MB 49.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 50 kB 5.7 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 1.4 MB 46.1 MB/s 
     |████████████████████████████████| 131 kB 44.5 MB/s 
     |████████████████████████████████| 3.8 MB 42.8 MB/s 
     |████████████████████████████████| 503 kB 48.6 MB/s 
     |████████████████████████████████| 7.9 MB 26.5 MB/s 
     |███████████████████████

In [53]:
!pip install rouge

In [1]:
# encoding text information usinf fasttext encoder
from embedding_as_service.text.encode import Encoder

In [2]:
# initialising fasttext encoder
fasttext_en = Encoder(embedding='fasttext', model='common_crawl_300')

1924it [00:00, 9723.28it/s]

Loading Model (this might take few minutes).....


1523723it [02:47, 9098.98it/s]

Model loaded Successfully !


In [3]:
glove_en = Encoder(embedding='glove', model='crawl_42B_300')

Model does not exists, Downloading model: crawl_42B_300
[||||||||||||||||||||||||||||||||||||||||||||||||||] 100 % [1833789/1833789 kb]


920it [00:00, 9190.63it/s]

Model downloaded successfully!
Loading Model (this might take few minutes).....


1917494it [03:37, 8808.07it/s]

Model loaded Successfully !


# Evaluation of the model

## Quantity Metrics

In [67]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [68]:
import numpy as np

# BLEU METRIC
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

# ROUGE METRIC
from rouge import Rouge

# JACCARD SIMILARITY
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer

# TFIDF SIMILARITY
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### BLEU SCORE

In [69]:
def BLEU_score(trans, ref ):
  '''
  This function helps in calculating BLEU score.
  '''
  smoothie = SmoothingFunction().method7
  score = sentence_bleu(ref, trans, smoothing_function=smoothie)
  print(f'\nBLEU SCORE : {score}')

### ROUGE SCORE

In [70]:
def ROUGE_score(trans, ref ):
  '''
  This function uses advance version of BLEU, and helps to calculate precision, recall and f1-score.
  '''
  rouge = Rouge()
  score = rouge.get_scores(trans,ref)
  print('\nROUGE SCORE')
  for i,j in score[0].items():
    print(i,j)

### JOCCARD SIMILARITY SCORE

In [71]:
def joccard_sim(trans, ref):
  '''
  This function calculates the Joccard Similarity by finding common words in both stories.
  '''
  lemmatizer = WordNetLemmatizer()
  translated_story_tokens = [lemmatizer.lemmatize(word.lower()) for word in word_tokenize(trans) if word not in stopwords]
  referal_story_tokens = [lemmatizer.lemmatize(word.lower()) for word in word_tokenize(ref) if word not in stopwords]

  common_words = len(set(translated_story_tokens).intersection(set(referal_story_tokens)))
  trans_words = len(set(translated_story_tokens))
  total_words = len(set(translated_story_tokens).union(set(referal_story_tokens)))

  print('\nJOCCARD SIMILARITY METRICS')
  print(f'Accuracy : {common_words/total_words*100:.4f}%')
  print(f'Precision : {common_words/trans_words*100:.4f}%')

### TFIDF SIMILARITY SCORE

In [72]:
def tfidf_sim(trans, ref):
  '''
  This function find cosine similarity between two stories based on the TFIDF Vectors encoding.
  '''
  tfidf_vec = TfidfVectorizer()

  corpus = [trans, ref]
  tfidf_vectors = tfidf_vec.fit_transform(corpus).todense()

  sim = cosine_similarity(tfidf_vectors[0], tfidf_vectors[1])

  print(f'\nTFIDF Similarity : {sim[0][0]*100:.4f} %.')

## Quality Metrics

### EMBEDDING SIMILARITY SCORE

In [73]:
def embedding_fasttext_sim(trans, ref):
  '''
  This function find cosine similarity between two stories based on the embedded vectors using FASTTEXT embedding.
  '''
  sim, final_sim = [],[]
  for translated_story_sent in sent_tokenize(trans):
    for referal_story_sent in sent_tokenize(ref):
      fasttext_vecs = fasttext_en.encode(texts = [translated_story_sent, referal_story_sent], pooling='reduce_mean')
      sim.append(cosine_similarity(fasttext_vecs[0].reshape(1, -1),fasttext_vecs[1].reshape(1, -1)))
    final_sim.append(max(sim)[0,0])
    sim=[]
  
  print(f'\nEMBEDDING FASTTEXT Similarity : {np.mean(final_sim)*100:.4f}%')

In [74]:
def embedding_glove_sim(trans, ref):
  '''
  This function find cosine similarity between two stories based on the embedded vectors using GLOVE embedding.
  '''
  sim, final_sim = [],[]
  for translated_story_sent in sent_tokenize(trans):
    for referal_story_sent in sent_tokenize(ref):
      fasttext_vecs = glove_en.encode(texts = [translated_story_sent, referal_story_sent], pooling='reduce_mean')
      sim.append(cosine_similarity(fasttext_vecs[0].reshape(1, -1),fasttext_vecs[1].reshape(1, -1)))
    final_sim.append(max(sim)[0,0])
    sim=[]
  
  print(f'\nEMBEDDING GLOVE Similarity : {np.mean(final_sim)*100:.4f}%')

In [75]:
def eval_text(trans, ref):
  '''
  This function helps in summarizing all metrics.
  INPUTS : Trans (Translated Story), ref (Referred Story)
  '''
  BLEU_score(trans, ref)
  ROUGE_score(trans, ref)
  joccard_sim(trans, ref)
  tfidf_sim(trans, ref)
  embedding_fasttext_sim(trans, ref)
  embedding_glove_sim(trans, ref)

# STORIES

In [76]:
# Translated Story by the model
translated_story = '''At the door of the hut both the father and the son lay buried, and inside, the young wife of the son, Budhia, was pining with pangs of labour. There was such a heart - wrenching sound coming out of his mouth that both of them held the collage together. It was a winter night, nature was in a state of silence, the whole village was in darkness. Ghisu said, I know it will not survive. I kept running all day long, come see. Madhav said angrily, If you have to die, why don \'t you die soon? What to see? You are toh very ajeeb! Such was the infidelity of the one with whom he or she had a good relationship for a whole year! "" "I don\'t see him shaking his hands and legs." There was a family of Chamars and a bad name in the whole village. If Ghisu worked one day, he would rest for three days. Madhav was so busy that if he worked for half an hour, he would drink for an hour. They were not getting wages for their work. If there was even a handful of grain in the house, he had sworn to work for them. When there were two or four holes, Ghisu would climb up the tree and break the wood and Madhav would sell it in the market. As long as the money remained, both of them wandered about. There was no work in the village. There was a village of farmers, there were fifty jobs for the hard working man. But these two would be called when there was no choice but to be satisfied with the work of one of the two men. If both were sadhus, they would not need at all restraint and discipline, for satisfaction and patience. This was their nature. It was a strange life! There was nothing in the house but two or four earthen pots. They lived by covering their nudity with torn cheths. Free From the Anxieties of the World! in debt. They also used to abuse, beat, and injure one another, but that did not stop them. Even when there was no hope of recovering the amount, people lent him some or some other loan. The pea, in the potato crop, would pluck the pea or potato from the fields of others and roast it and eat it or pluck the other five eyes and suck it at night. Ghisu spent sixty years of his life in the sky, and Madhav, like a son, was following in his father \'s footsteps, revealing his name even more. At this time, both of them were sitting in front of the bonfire, roasting potatoes, which they had dug out from a field. Ghisu \'s wife had died a long time ago. Madhav was married last year. From the time this woman came, she had laid the foundation of the system in this family and these two irregularities were a disgrace to India. When he arrived, they both wanted more rest. But there were some glitches too. If he was invited to do something, he would ask for a double wage. The same woman was dying of labour today, and the two of them were probably waiting for her to go to bed when she died. Ghisu plucked the potatoes and said, Go and see what his condition is. There will be a witch hunt, and what! Here even the carpenter is asking for a penny! Madhav was afraid that if he entered the room, Ghisu would clean up a large part of the potato. I am afraid to go there. What am I afraid of? Go and see it. When my wife was dead, I did not move from her for three days and would I be ashamed of myself again? Which I have never seen, today I see his mutilated body. He doesnt even have to look good. If you see me, you will not even be able to shake your hands and feet open! What happens if a child is born? Theres nothing at home! Everything will come. God bless you! Those who are not giving a single penny, they will call tomorrow and give Rs. I had nine sons, and there was never anything in the house, and God somehow or the other took it across. It was no wonder that in a society where the conditions of those who say day and night were no better than those who know how to take advantage of the peasant \'s weaknesses than those who know how to take advantage of them, such an attitude was born. We will say that Ghisu was more thoughtful to the peasants and instead of joining the group of farmers, the meeting took place in the ugly circle of barbers. Yes, it did not have the power to follow the rules and regulations of the sittings. So while the rest of his congregation were made head and chief of the village, the whole village pointed fingers at him. Still, he had the perception that if they were broken, he would not have to be as heartless as the peasant, and that his simplicity and inefficiency would not be exploited by others. Both the potatoes came out and started eating hot. I had not eaten anything since yesterday. They did not have to wait long for it to cool down. On several occasions, the tongues of both men burned. The outer part of the potato would not look very warm when chewed, but as soon as it fell under both the teeth, the inside of the potato burned the tongue, the bark, and the palate, and it was better to put the fire in the mouth than to put it in the mouth. There was enough to cool him down. Hence, both of them drowned. However, he had tears in his eyes in this attempt. Ghisu then remembered Tagore \'s quote in which he had gone twenty years ago. The satisfaction he gained at that feast was something to remember in his life, and his memory is still fresh in his mind. He does not forget the feast. There has been no food like that since then. The ladki had given them a full meal, everyone! All of them, young and old, ate pudis and real ghee. Chutney, Rita, three types of dried saag, a juicy vegetable, curds, chutneys, sweets, what the taste of that bhog was, there was no restriction, ask what you want, eat as much as you like. The people did eat, and the people did eat, and were not given water to drink. But those who are about to serve put hot, round, aromatic kachodis in the leaf. They feel that they are holding the leaf by hand, but they are given and when all of them have washed their faces, the betel - nut is also found. But how good it was to have a drink! He was not ruffled. She immediately went and lay down on her sofa. He was such a brave heart! \'No one now feeds us with such food, \'said Madhav, amused by the contents. "" "What will anyone eat?" "" That was the second time. Now everyone thinks it is affordable. Do not spend money on marriages. Dont waste your time on work. Ask me, where do you keep the poor \'s money? There is no work to do. Yes, it is cost - effective. \'You must have eaten at least twenty pies? I ate more than 20! I would have eaten fifty! I would not have eaten less than fifty. It was a good pitch. You are not even half of me. After eating the potatoes, the two of them drank water and lay down in front of the Alawa, covered with their dhoti and put their feet on the stomach. like two big python \'s have been killed. And Budhia was still crying. In the morning Madhav went to the room and found that his wife was cold. His face was soaked with flies. His stony eyes hung up. The body was covered with a quilt. The baby died in the womb. Madhav came to the fleeing Ghisu. Then both of them are loud yes and started beating their chests. On hearing the cries of the neighbours, they rushed to the spot and tried to explain the misfortunes according to the old norms. But there was not much to cry about. We had to worry about the coffins and the wood. Money was missing in the house, like a mouse in a eagles nest! Father and son went weeping to the zamindar of the village. He hated both of them. They had assaulted him several times. To steal, not to come to work on the fence. Why are you crying? he asked. You can\'t see it anymore! It seems that he does not want to live in this village. With his head on the ground, Ghisu said with tears in his eyes, Government! Im in very big trouble. Madhav \'s housekeeper passed away. The government was silent all night long. We were seated next to him. The drug addict did everything he could, but it betrayed us. Now there is not a single loaf of bread left to the master! were destroyed. The house was gutted. I am your servant, now who will cross its soil except you. Everything that we had in our hands got into our hands. If the government had the compassion, it would have been wiped out. Who else can I go to? Zamindar Sahib was kind. But to have mercy on Ghisu was to paint on a black blanket. Come here, tell me, get out of here. It \'s not even a call to the gun, but today when the thunder falls, he comes and rejoices. You bastards, bitch! But this was no occasion for anger or punishment. He took out two rupees and threw them away. But not a word of comfort came out of their mouths. He didnt even look at it. It is like removing the burden of the head. When Zamindar Saheb had given two rupees, how could the village banis and mahajans have the courage to refuse? Ghisu also knew how to beat the landlord \'s drum. One gave two, the other four. In an hour, Ghisu and Madhav went to fetch the kafan from the market. People started gathering here. The soft-hearted women of the village used to come and see the dead bodies, and two drops of tears flowed over his helplessness. When he reached the market, Ghisu said, The wood has got enough to burn it, why Madhav! Madhav said, Yes, the wood is plentiful, now you want kafan. "" "So let\'s take a little sip." Ya, right! The dead body will rise in the evening. Who watches the night? It is a bad custom that a person who does not get even a scratch to cover his body while living should have a new coffin at the time of death. The \'kafan\' goes with the dead body only in water. What is\'? \' If I had got five rupees here earlier, I would have got some medicine. They both stared at each other. They kept roaming around in the market. Sometimes Bajaj went to that shop, sometimes to that shop! He saw all kinds of clothes, silk and cotton, but nothing matched. even in the evening, the two of them, with a kind gesture, came to the front of a bar and went inside, as if by a predetermined arrangement, where they stood in confusion for a while, and then Ghisu went to the mound and said, Sir, give us a bottle too. Then came some vinegar, and some fried fish, and sat in the verandah, and drank in peace. After drinking a lot of Kujis came to Surur. Ghisu said, What does it mean to wear a kafan? After all, the water would go away, not go with some of the plum. Madhav looked up to the sky and said, It was as if the gods were being made to witness their own indestructibility. It is the tradition of the world. Why do people give thousands of rupees to the bats? Who will see whether in the future or not? The older men have money, blow it up, what do we have to blow it up? But what will people say? People will not ask, where is Kafan? "" "I will say that the money has slipped from the kammar." People will not believe it, but they will give the same amount. Madhav laughed at these unexpected fortunes and said, It was very good. Death is an evil foam! Half of the bottle flew away. Ghisu ordered two Ser \'s. Chutney, pickle, colleges. there was a shop in front of the bar. Madhav leapt up and brought all the stuff in two leaves. The whole dead rupee was spent. Only a little money was left. Both of them were sitting in the forest, eating with pride as if a lion was eating its prey. There was no fear of accountability, no fear of stigma. They had captured all these feelings long ago. Madhav bowed his head in reverence and said, Of course you must be. God, you are intuitive. Take him to Bykunth. We are both blessing each other with our hearts. The food that we get today was not available for a lifetime. After a moment, a doubt arose in Mann. He said, Why don \'t we also go there someday? Ghisu did not answer this naive question. He did not want to hinder this joy by thinking about the afterlife. "" "Those who asked us there, \'Why didn\'t you give us the coffin\', what would you say?" They will say \'your head\'! Ask me! How do you know he will not get the kafan? Do you think I\'m Jesus\' ass? Have I been digging the grass in the world for 60 years? He\'ll get the kafan and he\'ll get the best! Madhav could not believe it. Who said, who said? You have taken away Rs. He will ask me. I had put Sindur in his demand. Ghisu turned warm and said, I tell you, why don \'t you believe that he will get kafan? Why doesnt anyone say anything? "" "The people who have given it will give it to you." Yes, we will not have the money now. As darkness grew and the stars shone brightly, so did the roar of the madrasah. Some sang, some danced, some hugged their mates. Someone would put a razor in the mouth of a friend. There the atmosphere was warm, the air intoxicating. Some people would come here and go into a chimney. The air was more intoxicating than the wine. Barriers to life dragged on here and for a while they forgot whether they lived or died. They dont live or die. And these two father and son were still breathing and enjoying themselves. Everyones eyes had settled on them. How fortunate they are both! The whole bottle is in the middle. Having eaten enough, Madhav picked up the leaves of the leftovers and gave them to a beggar who was standing there watching from their hungry eyes. And for the first time in his life, he experienced the glory, the joy, and the exultation of giving. Gheesu said, Take it, eat well and bless. The wife, who had earned it, died. But your blessings will reach him. Bless me with weeping, the big car driver has money! Madhav looked up to the sky and said, She will go to Vaikunth Dada, she will become the queen of Vaikunth. Ghisu stood up and said, floating in the waves of jubilation, \'Yes, Beta, will go to Vaikuntha. No one was hurt and no damage occurred. Our greatest desire in life has been fulfilled by death. If she does not go, will the fat people go, who loot the poor with both hands, who bathe in the Ganges to wash their sins and offer water in the temples? This colour of devotion immediately changed. Instability is a characteristic of drugs. There was frustration and disappointment. Madhav said, But Grandpa, the poor girl had suffered a lot in her life. How tragic! He put his hand on his eyes and started crying. Ghisu explained why, son, be happy that she was free from the maze of magic and was free from the maze. I was lucky enough to break the bond of illusion so quickly. And the two of them stood up and began to sing: Thiagini naina jhamkave! The cheats! The eyes of the pikkaddas were fixed on them and they were both sung in their hearts. After which they both started laughing. Jump, even jump. He fell, he collapsed. He also composed and acted in plays, and eventually got drunk and fell there.'''

In [77]:
# already translated story used as a referral
end_goal = '''At the door of the hut father and son sat silently by a burnt-out fire; inside, the son's young wife Budhiya lay in labor, writhing with pain. And from time to time such a heart-rending scream emerged from her lips that they both pressed their hands to their hearts. It was a winter night; everything was drowned in desolation; The whole village had been absorbed into the darkness.

Ghisu said, "It seems she won't live. She's been writhing in pain the whole day. Go on-- see how she is."

Madhav said in a pained tone, "If she's going to die, then why doesn't she go ahead and die? What's the use of going to see?"

"You’re pretty hard-hearted! You’ve enjoyed life with her for a whole year-- such faithlessness to her?"

"Well, I can't stand to see her writhing and thrashing around."

It was a family of Chamars, and notorious in the whole village. If Ghisu worked for one day, then he rested for three. Madhav was such a slacker that if he worked for an hour, then he smoked his chilam for an hour. Thus nobody hired them on. If there was even a handful of grain in the house, they both swore off working. When they'd fasted for a couple of days, then Ghisu climbed trees and broke off branches, and Madhav sold the wood in the market; and as long as that money lasted, they both spent their time wandering idly around. *When their hunger grew intense, they again broke off branches, or looked for some work.* There was no shortage of work in the village. It was a village of farmers; for a hard-working man there were fifty jobs. But people only sent for those two when they were forced to content themselves with getting out of two men the work of one.

If only the two had been ascetics, then they wouldn't have needed any exercises in self-discipline to achieve contentment and patience. This was their very nature. Theirs was a strange life. Except for two or three clay pots, they had no goods at all in the house. Covering their nakedness with torn rags, free from the cares of the world, laden with debt-- they suffered abuse, they suffered blows too, but not grief. They were so poor that without the smallest hope of repayment, people used to lend them something or other. When peas or potatoes were in season, they would dig up peas or potatoes from the fields and roast and eat them, or break off five or ten stalks of sugarcane and suck them at night. Ghisu had spent sixty years of his life in this pious manner, and Madhav, like a dutiful son, was following in his father's footsteps-- or rather, was making his name even more radiant.

This time too, both were seated by the fire, roasting potatoes that they had dug up from somebody's field. Ghisu's wife had passed away long ago. Madhav's marriage had taken place the year before. Since this woman had come, she had laid the foundations of civilization in the family. *Grinding grain, cutting grass, she arranged for a couple of pounds of flour,* and kept filling the stomachs of those two shameless ones. After she came, they both grew even more lazy and indolent; indeed, they even began to swagger a bit. If someone sent for them to work, then with splendid indifference they demanded double wages. That woman was dying today in childbirth. And these two were perhaps waiting for her to die, so they could sleep in peace.

Pulling out a potato and peeling it, Ghisu said, "Go see what shape she's in. We'll have the fuss over a ghost-witch-- what else! And here even the exorcist demands a rupee-- *from whose house would we get one?*"

Madhav suspected that if he went into the hut, Ghisu would finish off most of the potatoes. He said, "I'm afraid to go in."

"What are you afraid of? I'm here, after all."

"Then you go and see, all right?"

"When my wife died, for three days I never even left her side. And then, won't she be ashamed in front of me? I've never seen her face-- and today I should see her naked body? She won't even have bodily ease: if she sees me, she won't be able to thrash around freely."

"I'm thinking, if a child is born-- what then? Dried ginger, brown sugar, oil-- there's nothing at all in the house."

"Everything will come. If Bhagwan gives a child-- those people who now aren't giving a paisa, will send for us and give us things. I've had nine sons. There was never anything in the house, but this is how we managed every time."

A society in which those who labored night and day were not in much better shape than these two; a society in which compared to the peasants, those who knew how to exploit the peasants' weaknesses were much better off-- in such a society, the birth of this kind of mentality was no cause for surprise. We'll say that compared to the peasants, Ghisu was more insightful; and instead of joining the mindless group of peasants, he had joined the group of clever, scheming tricksters. Though indeed, he wasn't skilful in following the rules and customs of the tricksters. Thus while other members of his group became chiefs and headmen of villages, at him the whole village wagged its finger. But still, he did have the consolation that if he was in bad shape, at least he wasn't forced to do the back-breaking labor of the peasants, and others didn't take improper advantage of his simplicity and voicelessness.

Pulling out the potatoes, they both began to eat them burning hot. They had eaten nothing since the day before. They were too impatient to wait till the potatoes cooled. Both burned their tongues repeatedly. When the potatoes were peeled, their outer parts didn't seem so extremely hot. But the moment the teeth bit into them, the inner part burned the tongue and throat and roof of the mouth. Rather than keep that ember in the mouth, it was better to send it quickly along inward, where there was plenty of equipment for cooling it down. So they both swallowed very fast, although the attempt brought tears to their eyes.

Then Ghisu remembered a landowner's wedding procession, in which he had taken part twenty years before. The repletion that had been vouchsafed to him in that feast was a memorable event in his life, and even today its memory was fresh. He said, "I'll never forget that feast. Never since then have I had that kind of food, or such a full stomach. The girl's family fed puris to everyone. As much as they wanted! Great and small, everyone ate puris-- ones made with real ghi! Chutney, raita, three kinds of green vegetables, a flavorful stew, yoghurt, chutney, sweets. How can I tell you now what relish there was in that feast! There was no limit. Whatever thing you want, just ask! And however much you want, eat! People ate so much, ate so much, that nobody could even drink any water. And there the servers were-- setting hot, round, sweet-smelling pastries before you! You refuse, saying you don't want it. You push away the tray with your hand. But that's how they are-- they just keep on giving it. And when everybody had wiped their mouths, then everybody got a pan as well. But how could I be in any shape for a pan? I couldn't stand up. I just staggered off and lay down on my blanket. He had a heart as big as the ocean, that landowner!"

Enjoying the story of these grand festivities, Madhav said, "If only somebody would give us such a feast now!"

"As if anybody would feast anybody now! That was a different time. Now everybody thinks about economy-- 'don't spend money on weddings, don't spend money on religious festivals!'. Ask them-- what's this 'saving' of the poor people's wealth? There's no lack of 'saving'. But when it comes to spending, they think about economy!"

"You must have eaten twenty or so puris?"

"I ate more than twenty."

"I would have eaten up fifty."

"I couldn't have eaten less than fifty. I was hale and hearty. You're not half of what I was!"

After eating, they both drank some water, covered themselves with their dhotis, curled up, and went to sleep right there by the fire, as if two gigantic serpents lay coiled there.

And Budhiya was still moaning.

In the morning, when Madhav went into the hut and looked, his wife had grown cold. Flies were buzzing on her face. Her stony eyes had rolled upward. Her whole body was covered with dust. In her stomach, the baby had died.

Madhav came running to Ghisu. Then they both together began loudly lamenting and beating their breasts. When the neighbors heard the weeping and wailing, they came running. And following the ancient custom, they began to console the bereaved.

But this wasn't the occasion for an excessive show of grief. They had to worry about the shroud, and the wood. Money was as scarce in their house as meat in a raptor's nest.

Father and son went weeping to the village landlord. He hated the very sight of their faces. A number of times he had beaten them with his own hands-- for theft, or for not coming to work as they had promised. He asked, "What is it, Ghisua, why do you weep? Nowadays we don't even see you around. It seems that you no longer want to live in the village."

Ghisua fell prostrate on the ground, and said with tear-filled eyes, "Master, I'm in great trouble! Madhav's wife passed away last night. All day she was writhing in pain, Master; we two sat by her bed till midnight. Whatever medicines we could give her, we did. But she slipped away. Now we have no one to care for us, Master-- we're devastated-- our house is destroyed! I'm your slave. Now who but you will take care of her final rites? Whatever money we had at hand was used up on medicines. If the Master will show mercy, then she'll have the proper rites. To whose door should I come except yours?"

The Landlord Sahib was a compassionate man. But to show compassion to Ghisu was to try to dye a black blanket. He felt like saying, "Get out of here! *Keep the corpse in your house and let it rot!* Usually you don't come even when you're called-- now when you want something, you come and flatter me! You treacherous bastard! You villain!" But this was not the occasion for anger or revenge. Willingly or not, he pulled out two rupees and flung them down. But he didn't open his lips to say a single word of consolation. He didn't even look in Ghisu's direction-- as if he'd discharged a duty.

When the Landlord Sahib gave two rupees, then how could the village merchants and money-lenders have the nerve to refuse? Ghisu knew how to beat the drum of the landlord's name. One gave two paisas, another gave four paisas. In an hour, Ghisu had collected the sum of five rupees in ready cash. Someone gave grain, someone else gave wood. And in the afternoon Ghisu and Madhav went to the market to get a shroud. Meanwhile, people began to cut the bamboo poles, and so on.

The sensitive-hearted women of the village came and looked at the body. They shed a few tears at its helplessness, and went away.

When they reached the market, Ghisu said, "We've got enough wood to burn her, haven't we, Madhav?"

Madhav said, "Yes, there's plenty of wood. Now we need a shroud."

"So let's buy a light kind of shroud."

"Sure, what else! While the body is being carried along, night will come. At night, who sees a shroud?"

"What a bad custom it is that someone who didn't even get a rag to cover her body when she was alive, needs a new shroud when she's dead."

"After all, the shroud burns along with the body."

"What else is it good for? If we'd had these five rupees earlier, we would have given her some medicine."

Each of them inwardly guessed what the other was thinking. They kept wandering here and there in the market, until eventually evening came. [Sometimes they went to one cloth-seller's shop, sometimes to another. They looked at various kinds of fabric, they looked at silk and cotton, but nothing suited them.] The two arrived, by chance or deliberately, before a wine-house; and as if according to some prearranged decision, they went inside. For a little while they both stood there in a state of uncertainty. [Then Ghisu went to the counter and said, "Sir, please give us a bottle too."] *Ghisu bought one bottle of liquor, and some sesame sweets.* [After this some snacks came, fried fish came]. And they both sat down on the verandah and [peacefully] began to drink.

After drinking a number of cups in a row, both became elevated.

Ghisu said, "What's the use of wrapping her in a shroud? After all, it would only be burned. Nothing would go with her."

Looking toward the sky as if persuading the angels of his innocence, Madhav said, "It's the custom of the world-- why do these same people give thousands of rupees to the Brahmins? Who can tell whether a reward does or doesn't reach them in another world?"

"Rich people have wealth-- let them waste it! What do we have to waste?"

"But what will you tell people? Won't people ask where the shroud is?"

Ghisu laughed. "We'll say the money slipped out of my waistband-- we searched and searched for it, but it didn't turn up. [People won't believe it, but they'll still give the same sum again.]"

Madhav too laughed at this unexpected good fortune, *at defeating destiny in this way*. He said, "She was very good, the poor thing. Even as she died, she gave us a fine meal."

More than half the bottle had been finished. Ghisu ordered two sers of puris, a meat stew, and spiced liver and fried fish. There was a shop right next to the wine-house. Madhav ran over and brought everything back on two leaf-plates. The cost was fully one and a half rupees. Only a few paise were left.

Both then sat eating puris, with all the majesty of a tiger in the jungle pursuing his prey. They had no fear of being called to account, nor any concern about disgrace. They had passed through these stages of weakness long ago. Ghisu said in a philosophical manner, "If my soul is being pleased, then won't she receive religious merit?"

Madhav bowed his head in pious confirmation. "Certainly she'll certainly receive it. Bhagwan, you are the knower of hearts-- take her to Heaven! We're both giving her our heartfelt blessing. The feast I've had today-- I haven't had its equal in my whole life!"

After a moment a doubt arose in Madhav's heart. He said, "How about it-- we'll go there too someday, won't we?"

Ghisu gave no answer to this childish question. *He looked reproachfully at Madhav.* [He didn't want the thought of heavenly matters to interfere with this bliss.]

"When she asks us, there, why we didn't give her a shroud, what will you say?"

"Oh, shut up!"

"She'll certainly ask."

"How do you know that she won't get a shroud? Do you consider me such a donkey? I've lived in this world for sixty years-- and have I just been loitering around? She'll get a shroud, and [a very good one]-- *a much better than we would have given*."

Madhav was not convinced. He said, "Who will give it? You've gobbled up the rupees! [It's me she'll ask-- I'm the one who put the sindur in the parting of her hair.]"

Ghisu grew irritated. "I tell you, she'll get a shroud. Why don't you believe me?"

"Who will give the money-- why don't you tell me?"

"The same people will give it who gave it this time. But they won't put the rupees into our hands. *And if somehow we get our hands on them, we'll sit here and drink again just like this, and they'll give the shroud a third time.*"

As the darkness deepened and and the stars glittered more sharply, the tumult in the wine-house also increased. One person sang, another babbled, another embraced his companion, another pressed a glass to his friend's lips. Joy was in the atmosphere there. Intoxication was in the air. How many people become 'an ass with a glass'! *They came here only to taste the pleasure of self-forgetfulness.* More than liquor, the air here elevated their spirits. The disaster of life seized them and dragged them here. And for a while they forgot whether they were alive or dead-- or half-alive.

And these two, father and son, were still sipping with relish. Everyone's eyes had settled on them. How fortunate they were! They had a whole bottle between them.

After he had finished eating, Madhav picked up the leaf-plate of leftover puris and gave it to a beggar who was standing there looking at them with hungry eyes. And for the first time in his life he felt the pride and delight and thrill of giving.

Ghisu said, "Take it-- eat your fill, and give her your blessing. She whose earnings these are has died, but your blessing will certainly reach her. Bless her with every hair on your body-- these are the payment for very hard labor."

Madhav again looked toward the sky and said, "She'll go to Heaven-- she'll become the Queen of Heaven!"

Ghisu stood up, and as if swimming in waves of joy he said, "Yes, son, she'll go to Heaven! She never tormented anyone, she never oppressed anyone; even while dying, she fulfilled the greatest desire of our lives. If she doesn't go to Heaven, then will those fat rich people go-- who loot the poor with both hands, and go to the Ganges to wash away their sin, and offer holy water in temples?"

This mood of piety too changed; variability is the special quality of intoxication. It was the turn of despair and grief. Madhav said, "But the poor thing suffered a great deal in her life. Even her death was so painful!" Covering his eyes with his hands, he began to weep, [and sobbed loudly].

Ghisu consoled him: "Why do you weep, son? Be happy that she's been liberated from this net of illusion. She's escaped from the snare; she was very fortunate that she was able to break the bonds of worldly illusion so quickly."

And both, standing there, began to sing, "Temptress! Why do your eyes flash, temptress?"

The whole wine-house was absorbed in the spectacle, and these two drinkers, deep in intoxication, kept on singing. Then they both began to dance-- they leaped and jumped, fell down, flounced about, gesticulated, [strutted around]; and finally, overcome by drunkenness, they collapsed.'''

In [78]:
translated_story

'At the door of the hut both the father and the son lay buried, and inside, the young wife of the son, Budhia, was pining with pangs of labour. There was such a heart - wrenching sound coming out of his mouth that both of them held the collage together. It was a winter night, nature was in a state of silence, the whole village was in darkness. Ghisu said, I know it will not survive. I kept running all day long, come see. Madhav said angrily, If you have to die, why don \'t you die soon? What to see? You are toh very ajeeb! Such was the infidelity of the one with whom he or she had a good relationship for a whole year! "" "I don\'t see him shaking his hands and legs." There was a family of Chamars and a bad name in the whole village. If Ghisu worked one day, he would rest for three days. Madhav was so busy that if he worked for half an hour, he would drink for an hour. They were not getting wages for their work. If there was even a handful of grain in the house, he had sworn to work for

In [79]:
# fixation just to bring referral story to same context with translated story
referred_story = "".join(end_goal.split('\n'))
referred_story

'At the door of the hut father and son sat silently by a burnt-out fire; inside, the son\'s young wife Budhiya lay in labor, writhing with pain. And from time to time such a heart-rending scream emerged from her lips that they both pressed their hands to their hearts. It was a winter night; everything was drowned in desolation; The whole village had been absorbed into the darkness.Ghisu said, "It seems she won\'t live. She\'s been writhing in pain the whole day. Go on-- see how she is."Madhav said in a pained tone, "If she\'s going to die, then why doesn\'t she go ahead and die? What\'s the use of going to see?""You’re pretty hard-hearted! You’ve enjoyed life with her for a whole year-- such faithlessness to her?""Well, I can\'t stand to see her writhing and thrashing around."It was a family of Chamars, and notorious in the whole village. If Ghisu worked for one day, then he rested for three. Madhav was such a slacker that if he worked for an hour, then he smoked his chilam for an hour

In [80]:
eval_text(translated_story, referred_story)


BLEU SCORE : 0.40813359373708996

ROUGE SCORE
rouge-1 {'r': 0.38313253012048193, 'p': 0.4732142857142857, 'f': 0.42343541449607364}
rouge-2 {'r': 0.1735769501054111, 'p': 0.20455486542443063, 'f': 0.18779699180223267}
rouge-l {'r': 0.3718875502008032, 'p': 0.4593253968253968, 'f': 0.4110075405502237}

JOCCARD SIMILARITY METRICS
Accuracy : 33.0364%
Precision : 53.4731%

TFIDF Similarity : 92.2328 %.

EMBEDDING FASTTEXT Similarity : 81.3795%

EMBEDDING GLOVE Similarity : 93.2124%
